##### Global import

Make sure that the following packages in `requirements.txt` are installed.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import matplotlib.ticker as mticker


## 1. Corpus Analysis


##### Copia import

In [3]:
from copia import density, accumulation_curve, species_accumulation
from copia.utils import evenness, survival_ratio
from copia.hill import hill_numbers
from copia.plot import evenness_plot


##### Load and prepare Data abundance


This code loads the Excel files `prose.xlsx` and `poetry.xlsx` from the `data/` folder, adds a `source` column to label each entry as prose or poetry, and merges them into a single DataFrame `df_combined`. It then separately counts the number of witnesses per work for prose and for poetry, storing these abundance values in `prose_witness_counts` and `poetry_witness_counts`, respectively.

In [9]:
# Load data files
folder_path = "data/"

df_prose = pd.read_csv(folder_path+"prose.csv", sep=";", engine="python")
prose_witness_counts = df_prose.groupby("text_ID")["witness_ID"].count()

df_poetry = pd.read_csv(folder_path+"poetry.csv", sep=";", engine="python")
poetry_witness_counts = df_poetry.groupby("text_ID")["witness_ID"].count()

#### 1.1 Data distribution

This code creates a histogram comparing the distribution of witness counts for prose and poetry. It uses a logarithmic scale on the x-axis to accommodate the wide range of values. Separate histograms are computed for prose and poetry using `np.histogram`, and their bars are plotted side by side for visual comparison. The histogram bars for prose works are red, while those for verse works are blue. Finally, the figure is saved as `figs/witness_dist.png` and displayed.


In [ ]:
# Create histogram with side-by-side bars
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(10, 7), dpi=300)
ax.set_facecolor('white')
ax.grid(True, alpha=0.4, linestyle='-', linewidth=0.5, color='gray')
ax.set_axisbelow(True)

# Define bins on logarithmic scale
max_witnesses = max(prose_witness_counts.max(), poetry_witness_counts.max())
bins = np.logspace(np.log10(1), np.log10(max_witnesses + 1), 15)

# Calculate histograms manually
prose_hist, _ = np.histogram(prose_witness_counts, bins=bins)
poetry_hist, _ = np.histogram(poetry_witness_counts, bins=bins)

# Calculate bin centers and widths in log scale
bin_centers = np.sqrt(bins[:-1] * bins[1:])
bin_widths = bins[1:] - bins[:-1]

# Bar width and positions
bar_width = bin_widths * 0.5
prose_positions = bin_centers - bar_width / 2
poetry_positions = bin_centers + bar_width / 2

# Plot side-by-side bars
ax.bar(prose_positions, prose_hist, width=bar_width, 
       label='Prose', alpha=0.8, edgecolor='black', linewidth=0.5)
ax.bar(poetry_positions, poetry_hist, width=bar_width, 
       label='Poetry', alpha=0.8, edgecolor='black', linewidth=0.5)

# Apply logarithmic scale to X axis
ax.set_xscale('log')

# Format X axis ticks
ax.xaxis.set_major_formatter(mticker.ScalarFormatter())
ax.xaxis.set_minor_formatter(mticker.NullFormatter())
ax.tick_params(axis='x', which='minor', bottom=False)
ax.tick_params(axis='both', which='major', labelsize=20)

# Labels and legend
ax.set_xlabel('Number of Witnesses per Work (Log Scale)', fontsize=24)
ax.set_ylabel('Total Number of Works', fontsize=24)
ax.legend(loc='upper right', fontsize=25)

plt.tight_layout()
plt.savefig("figs/witness_dist.png")
plt.show()

#### 1.2 Evenness Plot

This script assesses the evenness of textual transmission for prose and poetry. It should be recalled that evenness measures how equitably transmission is distributed across texts, with values near 1 indicating a balanced distribution and lower values reflecting concentration around a few dominant works. Hill numbers, computed with `hill_numbers`, provide a diversity index that accounts for both the number of distinct texts and their relative abundances. These are then converted into evenness values with the `evenness` function and stored in a dictionary for plotting.  The resulting figure is saved as a PNG file.


In [ ]:
# Calculate evenness
emp_prose, est_prose = hill_numbers(prose_witness_counts, n_iter=1000, n_jobs=4)
evenness_prose = evenness(est_prose)

emp_poetry, est_poetry = hill_numbers(poetry_witness_counts, n_iter=1000, n_jobs=4)
evenness_poetry = evenness(est_poetry)


evenness_result_dict = {'Prose': evenness_prose, 'Poetry': evenness_poetry}

# Create enhanced plot
fig, ax = plt.subplots(figsize=(10, 7), dpi=300)
evenness_plot(evenness_result_dict, figsize=(10, 7), ax=ax)

ax.set_facecolor('white')
ax.grid(True, alpha=0.4, linestyle='-', linewidth=0.5, color='gray')
ax.set_axisbelow(True)
ax.set_xlabel('Diversity order (q)', fontsize=24)
ax.set_ylabel('Evenness', fontsize=24)
ax.set_title(' ', fontsize=26, fontweight='bold')
ax.tick_params(axis='both', which='major', labelsize=20)
legend = ax.legend(fontsize=25)

for line in ax.get_lines():
    line.set_linewidth(3.5)

# Add grid for better readability
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.8)


plt.tight_layout()
plt.savefig("figs/evennesspvsp.png")
plt.show()

## 2. Results on prose texts

#### 2.1 Unseen Species approach

This code estimates the survival rate of prose texts in our corpus of Church Fathers in the Iberian Peninsula using the Chao1 estimator. First, the `survival_ratio` function with `method='ichao1'` computes a bootstrap distribution of survival ratios, providing an upper-bound estimate of the original number of texts. Finally, the code creates a density plot of these survival ratios and saves it as "figs/ichaoboot_prose.png".

NB: This code is essentially identical to the one used for the all corpus (prose or poetry).

In [ ]:
# Calculate survival ratios using Chao1 estimator
wsurvival_all = survival_ratio(prose_witness_counts, method='ichao1', n_iter=5000, n_jobs=1)

# Create enhanced density plot
fig, ax = plt.subplots(figsize=(10, 7), dpi=300)
density(wsurvival_all, ax = ax, figsize=(10, 7), xlim=(0.1, 1.1))

ax.set_facecolor('white')
ax.grid(True, alpha=0.4, linestyle='-', linewidth=0.5, color='gray')
ax.set_axisbelow(True)

ax.set_yticklabels([])
ax.tick_params(axis='y', which='both', left=False)
ax.set_ylabel(None)
ax.spines['left'].set_visible(False)
ax.set_xlabel('Survival Ratio', fontsize=24)
ax.tick_params(axis='x', which='major', labelsize=20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_linewidth(1.5)
ax.set_ylabel('Density', fontsize=24, fontweight='bold')


for line in ax.get_lines():
    line.set_linewidth(2.5)
ax.grid(axis='x', alpha=0.3, linestyle='--', linewidth=0.8)

for text in ax.texts:
    text.set_fontsize(25)
    text.set_bbox(dict(boxstyle='round,pad=0.5', facecolor='white', 
                       edgecolor='gray', alpha=0.8, linewidth=1.5))


plt.tight_layout()
plt.savefig("figs/ichaoboot_prose.png")
plt.show()

##### Witnesses survival rates

##### Witnesses survival rates

This code estimates the survival rate of witnesses for prose texts in our corpus. First, the `survival_ratio` function with `method='minsample'` computes a bootstrap distribution of survival ratios based on the number of witnesses, providing an estimate of how many witnesses have survived relative to the original set. Finally, the code creates a density plot of these survival ratios and saves it as "figs/minsampleboot.png".
Line in red corresponds to the estimate, and dashed lines represent the confidence interval at 95%.

NB: This code is essentially identical to the one used for the all corpus (prose or poetry).

In [ ]:
dsurvival_all = survival_ratio(prose_witness_counts, method='minsample', n_iter=1000, n_jobs=1)

# Create enhanced density plot
fig, ax = plt.subplots(figsize=(10, 7), dpi=300)
density(dsurvival_all, ax = ax, figsize=(10, 7), xlim=(0, 0.4))
ax.set_facecolor('white')
ax.grid(True, alpha=0.4, linestyle='-', linewidth=0.5, color='gray')
ax.set_axisbelow(True)

ax.set_yticklabels([])
ax.tick_params(axis='y', which='both', left=False)
ax.set_ylabel(None)
ax.spines['left'].set_visible(False)
ax.set_xlabel('Survival Ratio', fontsize=24)
ax.tick_params(axis='x', which='major', labelsize=20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_linewidth(1.5)
ax.set_ylabel('Density', fontsize=24, fontweight='bold')


for line in ax.get_lines():
    line.set_linewidth(2.5)
ax.grid(axis='x', alpha=0.3, linestyle='--', linewidth=0.8)

for text in ax.texts:
    text.set_fontsize(25)
    text.set_bbox(dict(boxstyle='round,pad=0.5', facecolor='white', 
                       edgecolor='gray', alpha=0.8, linewidth=1.5))


plt.tight_layout()
plt.savefig("figs/minsampleboot_prose.png")
plt.show()



##### Species accumulation curve

This code generates a species accumulation curve for our prose texts. In ecological terms, an accumulation curve shows the number of species discovered as a function of sampling effort; here, it represents the number of witnesses discovered as a function of the number of texts sampled. First, the `species_accumulation` function calculates the accumulation up to a realistic maximum number of steps (`max_steps = 20000`). Then, the `accumulation_curve` function plots this curve, showing how the number of witnesses increases with additional texts. The plot is customized for readability and clarity and is saved as "figs/accumulation_curve.png". The violet point represents empirical data, the dashed line is the extrapolation curve, and the green shaded area illustrates the uncertainty range.

NB: This code is essentially identical to the one used for the all corpus (prose or poetry).

In [ ]:
# Set realistic maximum steps for the corpus
max_steps = 20000

# Calculate species accumulation
accumulation = species_accumulation(prose_witness_counts, max_steps=max_steps)

fig, ax = plt.subplots(figsize=(10, 7), dpi=300)
ax.set_facecolor('white')
ax.grid(True, alpha=0.4, linestyle='-', linewidth=0.5, color='gray')
ax.set_axisbelow(True)

# Plot accumulation curve
accumulation_curve(prose_witness_counts, accumulation, c0='C2', c1='C2',
                   xlabel='Texts', ylabel='Witnesses',
                   ax=ax, xlim=(0, max_steps), label='Accumulation')

# Enhance inset plot appearance
ax.xaxis.get_offset_text().set_fontsize(24)
ax.tick_params(axis='both', labelsize=20)
ax.tick_params(axis='y', which='minor', left=False)
ax.ticklabel_format(style='sci', axis='x', scilimits=(0, 0), useMathText=True)
ax.set_xlabel('Texts', fontsize=24, fontweight='bold')
ax.set_ylabel('Witnesses', fontsize=24, fontweight='bold')

# Increase line width for better visibility
for line in ax.get_lines():
    line.set_linewidth(3)

# Add grid for better readability
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.8)
plt.savefig("figs/accumulation_curve_prose.png")
plt.show()

#### 2.1 Birth Death approach

This section implements the birth-death process simulation for generating trees under the three phase scenario (Innovation, Reproduction, Inactivity).

We first load the configuration file that contains all parameters for the church fathers pretrained model simulation :

In [ ]:
from src.cli.inference import inference
from src.cli.config import Config
from pathlib import Path


config_cf = Config("config_church_fathers_pretrained.yml")

Execute the birth-death simulation with the following parameters:
- **Input data**: Church fathers prose dataset (`data/prose.csv`)
- **Model configuration**: Using pretrained church fathers parameters
- **Output directory**: Results will be saved in `results_BD/`
- **Separator**: Using semicolon (`;`) as CSV separator
- **Save model**: Disabled for this run

The simulation will generate the distribution of posterior parameters $(\Lambda, \lambda, \mu)$ from the statistics computed on the dataset, and save all the results in the output directory.


In [ ]:
idata = inference(
                  csv_file = "data/prose.csv",
                  generator = config_cf.generator,
                  stats = config_cf.stats,
                  prior = config_cf.prior,
                  backend = config_cf.backend,
                  dir = Path("results_BD/"),
                  csv_separator = ";",
                  save_model = False,
                 )

#### CLI equivalent

Here is the equivalent command that can be run directly from the terminal/command line:
```bash
 simmatree -c config_church_fathers_pretrained.yml infer -i data/prose.csv -o results_BD
```